In [1]:
import pandas as pd
import numpy as np

In [2]:
# j'importe le DataFrame title_basics
df_title_basics = pd.read_csv('title.basics.tsv\data.tsv', sep="\t", low_memory=False)

In [3]:
# je remplace les \N par des NaN
df_title_basics = df_title_basics.replace("\\N", np.nan)
df_title_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


In [4]:
# je créé une liste des films que je souhaite supprimer, tout ce qui n'est pas movie et pas short
filter_Movie_Short=df_title_basics[(df_title_basics['titleType']!='movie') & (df_title_basics['titleType']!='short')]   
drop_list_movie = list(filter_Movie_Short['tconst'])
len(drop_list_movie)

7357281

In [5]:
filter_Movie_Short.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
20354,tt0020666,tvEpisode,Barnacle Bill,Barnacle Bill,0,1930,NaN,8,"Animation,Comedy,Family"
20515,tt0020829,tvEpisode,Dizzy Dishes,Dizzy Dishes,0,1930,NaN,6,"Animation,Comedy,Family"
20840,tt0021166,tvEpisode,Mysterious Mose,Mysterious Mose,0,1930,NaN,6,"Animation,Comedy,Family"
21270,tt0021612,tvEpisode,Any Little Girl That's a Nice Little Girl,Any Little Girl That's a Nice Little Girl,0,1931,NaN,7,"Animation,Comedy,Family"
21313,tt0021655,tvEpisode,Betty Co-ed,Betty Co-ed,0,1931,NaN,6,"Animation,Comedy,Family"


In [6]:
# je créé une liste des films que je souhaite supprimer, tout ce qui sont pour adult
filter_adult=df_title_basics[(df_title_basics['genres'].str.contains('Adult', na=False)) | (df_title_basics['isAdult'] == 1)]
drop_list_Adult = list(filter_adult['tconst'])

In [7]:
# je joins mes deux listes dans un array et je m'assure qu'il n'y ai pas occurence 
# j'ai ainsi la liste de tous les films non souhaités, je vais pour l'appliquer à toutes les bdd
filter_join_movies = drop_list_movie + drop_list_Adult
filter_movies=np.unique(np.array(filter_join_movies))
filter_movies

array(['tt0020666', 'tt0020829', 'tt0021166', ..., 'tt9916850',
       'tt9916852', 'tt9916880'], dtype='<U10')

In [8]:
# je supprime tous les movies de la liste dans title.basics
df_title_basics_filtered = df_title_basics[~df_title_basics.tconst.isin(filter_movies)]


In [9]:
df_title_basics_filtered.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


In [10]:
df_title_basics_filtered.shape

(1459915, 9)

In [11]:
# je refait le csv
df_title_basics_filtered.to_csv("df_title_basics_repack.csv", sep="\t", index=False)

In [13]:
df_title_basics_filtered.titleType.value_counts()

short    861970
movie    597945
Name: titleType, dtype: int64

In [37]:
# j'importe title.akas.tsv 
df_title_akas = pd.read_csv('title.akas.tsv\data.tsv', sep="\t", low_memory=False)

In [38]:
# je remplace les \N par des NaN
df_title_akas = df_title_akas.replace("\\N", np.nan)
df_title_akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


In [39]:
# je renomme titleId en tconst pour faire par la suite la jointure
df_title_akas = df_title_akas.rename(columns={'titleId': 'tconst'})
df_title_akas.head()


,tconst,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


In [40]:
# je filtre pour ne concerver que les films francais et les orignaux (pour les VO)
filter1=df_title_akas[(df_title_akas['region'] == 'FR') | (df_title_akas['types'] == 'original')]
filter1.head()

,tconst,ordering,title,region,language,types,attributes,isOriginalTitle
6,tt0000001,7,Carmencita,NaN,NaN,original,NaN,1
8,tt0000002,1,Le clown et ses chiens,NaN,NaN,original,NaN,1
9,tt0000002,2,Le clown et ses chiens,FR,NaN,imdbDisplay,NaN,0
21,tt0000003,6,Pauvre Pierrot,NaN,NaN,original,NaN,1
23,tt0000003,8,Pauvre Pierrot,FR,NaN,imdbDisplay,NaN,0


In [41]:
# je supprime tous les movies de la liste dans filter1
df_title_akas_filtered = filter1[~filter1.tconst.isin(filter_movies)]
df_title_akas_filtered.shape

(1344639, 8)

In [42]:
# je refait le csv
df_title_akas_filtered.to_csv("df_title_akas_repack.csv", sep="\t", index=False)

In [43]:
# j'importe title.crew.tsv pour lui appliquer le filtre et je refait le csv
df_title_crew = pd.read_csv('title.crew.tsv\data.tsv', sep="\t")
df_title_crew_filtered = df_title_crew[~df_title_crew.tconst.isin(filter_movies)]
df_title_crew_filtered.to_csv("df_title_crew_repack.csv", sep="\t", index=False)

In [45]:
# j'importe title.principals.tsv pour lui appliquer le filtre et je refait le csv
df_title_principals = pd.read_csv('title.principals.tsv\data.tsv', sep="\t")
df_title_principals_filtered = df_title_principals[~df_title_principals.tconst.isin(filter_movies)]
df_title_principals_filtered.to_csv("df_title_principals_repack.csv", sep="\t", index=False)

In [46]:
# j'importe title.ratings.tsv pour lui appliquer le filtre et je refait le csv
df_title_ratings = pd.read_csv('title.ratings.tsv\data.tsv', sep="\t")
df_title_ratings_filtered = df_title_ratings[~df_title_ratings.tconst.isin(filter_movies)]
df_title_ratings_filtered.to_csv("df_title_ratings_repack.csv", sep="\t", index=False)